# The Cool Mammals Project - Visualization

## Bar plots

First we will create a bar plot to present the countries names associated with the mammal species. For that, we use [Plotly](https://plot.ly/).

In [1]:
#Import modules 
import numpy as np
import pandas as pd
import ast
import csv

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import plot, iplot, init_notebook_mode

# set plotly to display on notebook
init_notebook_mode(connected=True)

In [10]:
# Function to read from countries counts .csv file
def mammals_countries(mammal):    
    #Load and prepare data
    df = pd.read_csv('mammals_country_counts_v1.csv')
    npages = df.loc[df['species']==mammal]['number_results'].values[0]
    aux = df.loc[df['species']==mammal]['country_counts'].values[0]
    aux = aux.strip('[]')
    aux_all = list(ast.literal_eval(aux))

    #Check if it's a valid country name and counts occurrences
    with open('countries_names.csv', 'r') as f:
        reader = csv.reader(f)
        valid_countries = list(reader)[0]
    country_names = []
    country_counts = []
    for i in aux_all:   
        if i[0].title() in valid_countries:  
            country_names.append(i[0].title())
            country_counts.append(i[1])
    country_names = np.array(country_names)
    country_counts = 100*np.array(country_counts)/npages
    
    return country_names, country_counts


# function to generate bar plots
def bar_graph_mammals(mammal, cnt_threshold=5): 
    #Get countries counts
    country_names, country_counts = mammals_countries(mammal)
    if len(country_names) > 30:
        country_names = country_names[0:30]
        country_counts = country_counts[0:30]
    
    #Make Plotly graphics
    trace0 = go.Bar(
        x=country_names,
        y=country_counts,
        marker=dict( color='rgb(158,202,225)',
                     line=dict(color='rgb(8,48,107)', width=1.5) ),
        opacity=0.6 )

    data = [trace0]   
    
    layout = dict( height=370, width=900, xaxis=dict(tickangle=-45), title=mammal+' presence',
                   yaxis=dict( title='Occurrences on web pages [%]' ),
                   paper_bgcolor='rgba(0,0,0,0)',
                   plot_bgcolor='rgba(0,0,0,0)',
                   shapes=[{
                        'type': 'line',
                        'x0': 0,
                        'y0': cnt_threshold,
                        'x1': len(country_names),
                        'y1': cnt_threshold,
                        'line': {
                            'color': 'rgb(0, 0, 0)',
                            'width': 1,
                            'dash': 'dot'}}] )

    fig = go.Figure(data=data, layout=layout)
    
    #Display bar plots
    iplot(fig)
    
    #Save bar plots
    plot(fig, filename='media/bar_graph_'+mammal+'.html')
    
    return country_names, country_counts
        

## Maps

Maps are amongst the best engaging visual tool we can use when telling a story. Creating information-rich maps in python is very easy, thanks to [Folium](https://github.com/python-visualization/folium).
We will create a choropleth map, with color intensity indicating how strongly associated with a mammal species each country is.
The cute icons were taken from [Flaticon](https://www.flaticon.com/).

In [4]:
import folium

In [7]:
# function to generate map plots
def map_mammals(mammal, cnt_threshold, country_names, country_counts): 
    # make an empty map
    my_map = folium.Map(location=[20, 0], tiles="Mapbox Bright", zoom_start=2)
    #my_map = folium.Map(location=[20, 0], tiles="Stamen Terrain", zoom_start=2)

    #Countries layer
    countries_geo = 'world-countries.json'
    countries_df = pd.read_json(countries_geo)
    for index, row in countries_df.iterrows():
        countries_df.at[index,'country'] = row['features']['properties']['name']

    chosen_indexes = country_counts > cnt_threshold
    chosen_data = pd.DataFrame(data=country_names[chosen_indexes], columns=['country'])
    chosen_data['quantity'] = country_counts[chosen_indexes]

    # Add the color for the chloropleth
    folium.Choropleth(
        geo_data=countries_geo,
        name='choropleth',
        data=chosen_data,
        columns=['country', 'quantity'],
        key_on='properties.name', #'feature.id',
        fill_color='OrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        nan_fill_color ='#ffffff00',
        legend_name='Estimated presence of '+mammal+'s',
    ).add_to(my_map)

    #create mammal markers and add them to map object 
    centroids = pd.read_csv('centroids.csv')  #centroid marks to print icons    
    for index, row in chosen_data.iterrows():
        #create icons from images
        icon = folium.features.CustomIcon('./icons/icon_'+mammal+'.png', icon_size=(25,25))
        #create popup descriptions
        popupIcon = "<strong>mammal</strong><br>Population"
        if sum(centroids['country']==row.country) == 1:
            lat = centroids.loc[centroids['country']==row.country]['lat']
            lng = centroids.loc[centroids['country']==row.country]['lng']
            folium.Marker([lat.values[0],lng.values[0]], tooltip=mammal, popup=popupIcon, icon=icon).add_to(my_map)

    # Save to html
    my_map.save('media/map_'+mammal+'.html')
    
    return my_map

Now we call the functions as enjoy the nice graphics!

In [26]:
# Choose a species from the list and a limiting threshold to get rid of noise results
#
#mammal_list = ['hedgehog', 'lion', 'wolf', 'fox', 'zebra', 'giraffe',
#               'bat', 'sloth', 'capybara', 'elephant', 'rhino',
#               'hippo', 'tiger', 'panda', 'kangaroo', 'koala', 'human']

mammal = 'capybara'    
cnt_threshold = 2

country_names, country_counts = bar_graph_mammals(mammal, cnt_threshold = cnt_threshold) 
my_map = map_mammals(mammal, cnt_threshold=cnt_threshold, country_names=country_names, country_counts=country_counts) 
my_map